# B2B Travel Fraud Detection — Synthetic Data Generator

Each cell below generates one table. Run them **in order** (top → bottom).

**Tables**: `agency_master`, `user_master`, `booking_fact`, `session_context`, `passenger_details`, `post_booking_events`, `booking_label_table`

In [11]:
# ============================================================
# CELL 0 — IMPORTS & CONFIG
# ============================================================
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import random, os, zipfile, uuid, hashlib

random.seed(42)
np.random.seed(42)

N_BOOKINGS = 8000
N_USERS = 800
N_FRAUD_USERS = 40
TARGET_FRAUD_BOOKINGS = 720

N_AGENCIES = 200
START_DATE = datetime(2025, 1, 1)

OUT_DIR = os.path.join(os.path.dirname(os.getcwd()), "Data", "Tables")
os.makedirs(OUT_DIR, exist_ok=True)

# --- helpers -----------------------------------------------
def make_uuid():
    return str(uuid.uuid4())

def make_fingerprint(seed_str):
    """Deterministic hardware/cookie hash from a seed string."""
    return hashlib.sha256(seed_str.encode()).hexdigest()[:16]

def random_ip():
    return ".".join(map(str, np.random.randint(1, 255, size=4)))

# Datacenter ASN ranges — the ONLY way we detect VPN/proxy.
# If ASN falls in these ranges → is_vpn_or_proxy = 1, else 0.
DATACENTER_ASN_RANGES = set(range(10000, 15000)) | set(range(30000, 35000))

# --- Email domain pools ------------------------------------
CORPORATE_DOMAIN_POOL = [
    "travelcorp.com", "globeways.in", "flyhigh.ae", "skybound.co",
    "transwings.com", "jetsetgo.in", "voyagepro.sg", "airdesk.uk",
    "tripstation.com", "bookmyflight.in", "travease.ae", "routemaster.co",
    "flydeals.com", "wingspan.sg", "gotravel.in", "aerobiz.uk",
    "traveledge.com", "swiftfly.in", "jetstream.ae", "horizonair.co",
    "quicktrips.com", "flynext.sg", "tourhub.in", "skypaths.uk",
    "travelprime.com", "flybright.ae", "airsync.in", "globalwings.co",
    "travelworks.sg", "flyweb.com", "tripcraft.in", "aerolink.uk",
    "airventure.com", "flyscope.ae", "travelzen.in", "jetforce.co",
    "airnova.sg", "travelmax.com", "skyroute.in", "wingcraft.uk"
]

GENERIC_DOMAINS = [
    "gmail.com", "yahoo.com", "outlook.com", "hotmail.com",
    "protonmail.com", "mail.com", "yandex.com", "aol.com",
    "zoho.com", "icloud.com"
]

THROWAWAY_DOMAINS = [
    "tempmail.xyz", "fakeinbox.org", "trashmail.net", "guerrillamail.com",
    "mailinator.com", "throwaway.email", "discard.email", "sharklasers.com"
]

print("✅ Config loaded")

✅ Config loaded


In [12]:
# ============================================================
# CELL 1 — AGENCY MASTER
# ============================================================
agency_ids = [f"A{str(i).zfill(4)}" for i in range(1, N_AGENCIES + 1)]
agency_domains = [CORPORATE_DOMAIN_POOL[i % len(CORPORATE_DOMAIN_POOL)] for i in range(N_AGENCIES)]

agency_master = pd.DataFrame({
    "agency_id": agency_ids,
    "country": np.random.choice(["IN", "AE", "SG", "UK", "US"], size=N_AGENCIES,
                                p=[0.45, 0.15, 0.10, 0.15, 0.15]),
    "agency_age_days": np.random.randint(30, 3000, size=N_AGENCIES),
    "kyc_status": np.random.choice(["verified", "pending", "failed"], size=N_AGENCIES,
                                   p=[0.85, 0.12, 0.03]),
    "credit_limit": np.random.choice([5000, 10000, 25000, 50000, 100000, 200000, 500000],
                                     size=N_AGENCIES,
                                     p=[0.20, 0.20, 0.20, 0.15, 0.15, 0.07, 0.03]),
    "status": np.random.choice(["active", "suspended"], size=N_AGENCIES, p=[0.97, 0.03]),
    "agency_email_domain": agency_domains
})

agency_to_domain = dict(zip(agency_master["agency_id"], agency_master["agency_email_domain"]))

print(f"✅ agency_master — {len(agency_master)} rows")
print(f"   Sample domains: {agency_master['agency_email_domain'].head(5).tolist()}")
agency_master.head()

✅ agency_master — 200 rows
   Sample domains: ['travelcorp.com', 'globeways.in', 'flyhigh.ae', 'skybound.co', 'transwings.com']


,agency_id,country,agency_age_days,kyc_status,credit_limit,status,agency_email_domain
0,A0001,IN,657,verified,5000,active,travelcorp.com
1,A0002,US,616,verified,200000,active,globeways.in
2,A0003,UK,1678,verified,50000,active,flyhigh.ae
3,A0004,AE,1473,verified,10000,active,skybound.co
4,A0005,IN,1475,verified,50000,active,transwings.com


In [13]:
# ============================================================
# CELL 2 — USER MASTER (balanced fraud user types)
# ============================================================
user_ids = [f"U{str(i).zfill(5)}" for i in range(1, N_USERS + 1)]
user_agencies = np.random.choice(agency_ids, size=N_USERS)

fraud_user_ids = np.random.choice(user_ids, size=N_FRAUD_USERS, replace=False)
fraud_user_set = set(fraud_user_ids)

fraud_type_counts = {
    "account_takeover": 7, "ring_operator": 7, "bot_booking": 7,
    "cancellation_abuser": 6, "credit_bustout_user": 7, "new_synthetic_user": 6
}
assert sum(fraud_type_counts.values()) == N_FRAUD_USERS

balanced_fraud_types = []
for k, v in fraud_type_counts.items():
    balanced_fraud_types.extend([k] * v)
random.shuffle(balanced_fraud_types)
fraud_type_map = dict(zip(fraud_user_ids, balanced_fraud_types))

roles = np.random.choice(["agent", "admin", "finance"], size=N_USERS, p=[0.78, 0.12, 0.10])
user_age_days = np.random.randint(30, 2500, size=N_USERS)
avg_logins = np.clip(np.random.normal(6, 3.5, size=N_USERS), 0, 60)
failed_login_ratio = np.clip(np.random.beta(1.5, 40, size=N_USERS), 0, 1)
account_status = np.random.choice(["active", "locked"], size=N_USERS, p=[0.985, 0.015])
email_domain_match = np.zeros(N_USERS, dtype=int)

for i, uid in enumerate(user_ids):
    if uid in fraud_user_set:
        ftype = fraud_type_map[uid]
        if ftype == "account_takeover":
            failed_login_ratio[i] = np.random.uniform(0.18, 0.55)
            avg_logins[i] = np.random.uniform(10, 25)
        elif ftype == "bot_booking":
            failed_login_ratio[i] = np.random.uniform(0.05, 0.20)
            avg_logins[i] = np.random.uniform(25, 60)
        elif ftype == "ring_operator":
            failed_login_ratio[i] = np.random.uniform(0.03, 0.14)
            avg_logins[i] = np.random.uniform(12, 28)
        elif ftype == "cancellation_abuser":
            failed_login_ratio[i] = np.random.uniform(0.01, 0.10)
            avg_logins[i] = np.random.uniform(8, 18)
        elif ftype == "credit_bustout_user":
            failed_login_ratio[i] = np.random.uniform(0.01, 0.08)
            avg_logins[i] = np.random.uniform(10, 22)
        elif ftype == "new_synthetic_user":
            user_age_days[i] = np.random.randint(3, 40)
            failed_login_ratio[i] = np.random.uniform(0.02, 0.18)
            avg_logins[i] = np.random.uniform(12, 40)
        if np.random.rand() < 0.05:
            account_status[i] = "locked"

        if ftype in ["new_synthetic_user", "ring_operator"]:
            email_domain_match[i] = int(np.random.rand() < 0.15)
        elif ftype in ["bot_booking", "account_takeover"]:
            email_domain_match[i] = int(np.random.rand() < 0.35)
        elif ftype == "credit_bustout_user":
            email_domain_match[i] = int(np.random.rand() < 0.50)
        else:
            email_domain_match[i] = int(np.random.rand() < 0.75)
    else:
        email_domain_match[i] = int(np.random.rand() < 0.90)

user_master = pd.DataFrame({
    "user_id": user_ids, "agency_id": user_agencies, "role": roles,
    "user_age_days": user_age_days,
    "avg_logins_per_day": np.round(avg_logins, 2),
    "failed_login_ratio": np.round(failed_login_ratio, 4),
    "account_status": account_status,
    "email_domain_match_flag": email_domain_match,
    "user_fraud_label": [1 if u in fraud_user_set else 0 for u in user_ids],
    "user_fraud_type": [fraud_type_map[u] if u in fraud_user_set else "legit" for u in user_ids],
})

user_to_agency = dict(zip(user_master["user_id"], user_master["agency_id"]))

print(f"✅ user_master — {len(user_master)} rows")
print(f"   Fraud users: {user_master['user_fraud_label'].sum()}")
print(f"   Email domain match (legit): {user_master[user_master['user_fraud_label']==0]['email_domain_match_flag'].mean()*100:.1f}%")
print(f"   Email domain match (fraud): {user_master[user_master['user_fraud_label']==1]['email_domain_match_flag'].mean()*100:.1f}%")
print(user_master[user_master['user_fraud_label']==1]['user_fraud_type'].value_counts())
user_master.head()

✅ user_master — 800 rows
   Fraud users: 40
   Email domain match (legit): 93.2%
   Email domain match (fraud): 45.0%
user_fraud_type
account_takeover       7
bot_booking            7
credit_bustout_user    7
ring_operator          7
new_synthetic_user     6
cancellation_abuser    6
Name: count, dtype: int64


,user_id,agency_id,role,user_age_days,avg_logins_per_day,failed_login_ratio,account_status,email_domain_match_flag,user_fraud_label,user_fraud_type
0,U00001,A0100,agent,2033,1.05,0.0412,active,1,0,legit
1,U00002,A0041,agent,722,1.45,0.0058,active,1,0,legit
2,U00003,A0166,agent,1135,8.98,0.0110,active,1,0,legit
3,U00004,A0034,agent,2401,11.25,0.0500,active,1,0,legit
4,U00005,A0017,agent,2191,5.73,0.0243,active,1,0,legit


In [14]:
# ============================================================
# CELL 3 — BOOKING FACT
# ============================================================
booking_ids = [f"B{str(i).zfill(6)}" for i in range(1, N_BOOKINGS + 1)]
booking_ts = [START_DATE + timedelta(minutes=int(x)) for x in np.random.randint(0, 365 * 24 * 60, size=N_BOOKINGS)]

product_type = np.random.choice(["flight", "hotel", "package"], size=N_BOOKINGS, p=[0.6, 0.3, 0.1])
route_type = np.random.choice(["domestic", "international"], size=N_BOOKINGS, p=[0.65, 0.35])
origin_country = np.random.choice(["IN", "AE", "SG", "UK", "US"], size=N_BOOKINGS,
                                  p=[0.45, 0.15, 0.10, 0.15, 0.15])
dest_country = np.where(route_type == "domestic", origin_country,
                        np.random.choice(["IN", "AE", "SG", "UK", "US", "TH", "FR", "DE"], size=N_BOOKINGS))
lead_time_days = np.where(route_type == "domestic",
                          np.random.randint(1, 20, size=N_BOOKINGS),
                          np.random.randint(2, 90, size=N_BOOKINGS))

def gen_value(rt):
    if rt == "domestic":
        return float(np.clip(np.random.lognormal(mean=5.3, sigma=0.55), 40, 12000))
    return float(np.clip(np.random.lognormal(mean=7.2, sigma=0.65), 200, 25000))

booking_value = np.array([gen_value(rt) for rt in route_type])
passengers_count = np.random.choice([1,2,3,4,5,6,7,8,9,10], size=N_BOOKINGS,
                                    p=[0.35,0.25,0.15,0.08,0.05,0.04,0.03,0.025,0.015,0.01])
payment_method = np.random.choice(["credit_line", "card", "bank"], size=N_BOOKINGS, p=[0.75, 0.18, 0.07])
booking_status = np.random.choice(["confirmed", "pending", "failed"], size=N_BOOKINGS, p=[0.92, 0.06, 0.02])

fraud_booking_indices = np.random.choice(np.arange(N_BOOKINGS), size=TARGET_FRAUD_BOOKINGS, replace=False)
fraud_booking_set = set(fraud_booking_indices)

fraud_users_list = list(fraud_user_ids)
fraud_users_for_bookings = np.random.choice(fraud_users_list, size=32, replace=False)
legit_users_list = [u for u in user_ids if u not in fraud_user_set]

user_id_col = np.empty(N_BOOKINGS, dtype=object)
agency_id_col = np.empty(N_BOOKINGS, dtype=object)

for i in range(N_BOOKINGS):
    if i in fraud_booking_set:
        u = np.random.choice(fraud_users_for_bookings)
    else:
        u = np.random.choice(legit_users_list)
    user_id_col[i] = u
    agency_id_col[i] = user_to_agency[u]

booking_fact = pd.DataFrame({
    "booking_id": booking_ids, "booking_ts": booking_ts,
    "agency_id": agency_id_col, "user_id": user_id_col,
    "product_type": product_type, "route_type": route_type,
    "origin_country": origin_country, "dest_country": dest_country,
    "lead_time_days": lead_time_days.astype(int),
    "booking_value": np.round(booking_value, 2),
    "passengers_count": passengers_count.astype(int),
    "payment_method": payment_method, "booking_status": booking_status
})

print(f"✅ booking_fact — {len(booking_fact)} rows")
print(f"   Fraud bookings: {len(fraud_booking_set)}")
booking_fact.head()

✅ booking_fact — 8000 rows
   Fraud bookings: 720


,booking_id,booking_ts,agency_id,user_id,product_type,route_type,origin_country,dest_country,lead_time_days,booking_value,passengers_count,payment_method,booking_status
0,B000001,2025-07-26 02:24:00,A0072,U00567,hotel,domestic,IN,IN,6,205.91,6,credit_line,failed
1,B000002,2025-11-23 10:58:00,A0082,U00607,flight,domestic,US,US,13,79.82,1,card,confirmed
2,B000003,2025-01-29 02:51:00,A0105,U00272,package,domestic,IN,IN,19,332.24,6,credit_line,confirmed
3,B000004,2025-02-13 18:43:00,A0179,U00642,hotel,domestic,US,US,3,469.32,2,credit_line,confirmed
4,B000005,2025-03-31 06:48:00,A0124,U00448,hotel,domestic,IN,IN,16,221.84,3,credit_line,confirmed


In [15]:
# ============================================================
# CELL 4 — SESSION CONTEXT
# One row per booking. Replaces old device_master + ip_master.
#
# Each device_fingerprint is a physical device — same fp always
# maps to same OS + browser.
#
# SESSION PATTERNS:
#   1. Shared infra   — fraud users share FP/IP from fraud pool
#   2. Sticky infra   — user reuses their own pool of FP/IPs
#   3. IP MASKING      — same device (sticky FP) but RANDOM IP
#                        each session (VPN/proxy rotation).
#                        Creates "same device, many IPs" signal.
#
# VPN DETECTION RULE:
#   is_vpn_or_proxy is derived SOLELY from ASN.
#   If ASN ∈ DATACENTER_ASN_RANGES → 1, else → 0.
#   This is our only methodology to detect VPN/proxy usage.
#   Fraud users get higher VPN rates because:
#     - IP masking forces datacenter ASNs
#     - Infra-heavy fraud types have higher datacenter ASN prob.
# ============================================================

N_FINGERPRINTS = 500
N_IPS = 600

all_fingerprints = [make_fingerprint(f"dev_{i}") for i in range(N_FINGERPRINTS)]
all_ips = [random_ip() for _ in range(N_IPS)]

fraud_fp_pool = list(np.random.choice(all_fingerprints, size=20, replace=False))
normal_fp_pool = [f for f in all_fingerprints if f not in set(fraud_fp_pool)]

fraud_ip_pool = list(np.random.choice(all_ips, size=24, replace=False))
normal_ip_pool = [ip for ip in all_ips if ip not in set(fraud_ip_pool)]

# Pre-assign fixed OS + browser per fingerprint
os_choices = ["Windows", "macOS", "Android", "iOS"]
os_probs = [0.45, 0.20, 0.25, 0.10]
browser_choices = ["Chrome", "Safari", "Edge", "Firefox"]
browser_probs = [0.65, 0.12, 0.15, 0.08]

fp_to_os = {fp: np.random.choice(os_choices, p=os_probs) for fp in all_fingerprints}
fp_to_browser = {fp: np.random.choice(browser_choices, p=browser_probs) for fp in all_fingerprints}

# --- Sticky pools per user ---------------------------------
user_sticky_fps = {}
user_sticky_ips = {}

for u in legit_users_list:
    user_sticky_fps[u] = [np.random.choice(normal_fp_pool)]
    k_ips = np.random.choice([1, 2], p=[0.85, 0.15])
    user_sticky_ips[u] = list(np.random.choice(normal_ip_pool, size=k_ips, replace=False))

for u in fraud_users_list:
    utype = user_master.loc[user_master["user_id"] == u, "user_fraud_type"].values[0]
    if utype in ["ring_operator", "bot_booking"]:
        k_fp = np.random.choice([2, 3, 4], p=[0.30, 0.50, 0.20])
        k_ip = np.random.choice([3, 4, 5], p=[0.30, 0.50, 0.20])
    elif utype in ["account_takeover", "new_synthetic_user"]:
        k_fp = np.random.choice([2, 3], p=[0.60, 0.40])
        k_ip = np.random.choice([2, 3, 4], p=[0.40, 0.40, 0.20])
    elif utype == "cancellation_abuser":
        k_fp = np.random.choice([1, 2], p=[0.70, 0.30])
        k_ip = np.random.choice([1, 2], p=[0.70, 0.30])
    elif utype == "credit_bustout_user":
        k_fp = np.random.choice([1, 2], p=[0.60, 0.40])
        k_ip = np.random.choice([2, 3], p=[0.60, 0.40])
    else:
        k_fp, k_ip = 2, 2
    user_sticky_fps[u] = list(np.random.choice(all_fingerprints, size=k_fp, replace=False))
    user_sticky_ips[u] = list(np.random.choice(all_ips, size=k_ip, replace=False))

# --- IP masking probability per fraud type ------------------
IP_MASK_PROB = {
    "account_takeover":    0.35,
    "bot_booking":         0.30,
    "ring_operator":       0.25,
    "new_synthetic_user":  0.30,
    "credit_bustout_user": 0.10,
    "cancellation_abuser": 0.05,
    "legit":               0.00,
}

# --- Datacenter ASN probability per fraud type --------------
# Even without IP masking, infra-heavy fraud types are more
# likely to route through datacenters (shared hosting, etc.).
# Legit users rarely hit datacenter ASNs.
DATACENTER_ASN_PROB = {
    "account_takeover":    0.35,
    "bot_booking":         0.40,
    "ring_operator":       0.30,
    "new_synthetic_user":  0.30,
    "credit_bustout_user": 0.10,
    "cancellation_abuser": 0.05,
    "legit":               0.02,
}

DATACENTER_ASN_LIST = list(DATACENTER_ASN_RANGES)

# --- Generate one session row per booking -------------------
session_rows = []
ip_masked_count = 0

for i in range(N_BOOKINGS):
    u = booking_fact.loc[i, "user_id"]
    utype = user_master.loc[user_master["user_id"] == u, "user_fraud_type"].values[0]

    ip_masking = np.random.rand() < IP_MASK_PROB.get(utype, 0.0)

    if ip_masking:
        fp = np.random.choice(user_sticky_fps[u])
        ip = random_ip()
        ip_masked_count += 1
    elif u in fraud_user_set:
        if utype in ["ring_operator", "bot_booking"]:
            fp = np.random.choice(fraud_fp_pool) if np.random.rand() < 0.65 else np.random.choice(user_sticky_fps[u])
            ip = np.random.choice(fraud_ip_pool) if np.random.rand() < 0.65 else np.random.choice(user_sticky_ips[u])
        elif utype in ["account_takeover", "new_synthetic_user"]:
            fp = np.random.choice(fraud_fp_pool) if np.random.rand() < 0.55 else np.random.choice(user_sticky_fps[u])
            ip = np.random.choice(fraud_ip_pool) if np.random.rand() < 0.55 else np.random.choice(user_sticky_ips[u])
        else:
            fp = np.random.choice(user_sticky_fps[u])
            ip = np.random.choice(user_sticky_ips[u])
    else:
        fp = user_sticky_fps[u][0]
        ip = np.random.choice(user_sticky_ips[u])
        if np.random.rand() < 0.01:
            ip = np.random.choice(fraud_ip_pool)

    # --- ASN assignment ---
    # IP masking → always datacenter ASN
    # Non-masking → datacenter ASN based on fraud-type probability
    if ip_masking:
        asn = int(np.random.choice(DATACENTER_ASN_LIST))
    elif np.random.rand() < DATACENTER_ASN_PROB.get(utype, 0.02):
        asn = int(np.random.choice(DATACENTER_ASN_LIST))
    else:
        # Non-datacenter ASN: pick from ranges that DON'T overlap
        asn = int(np.random.choice(
            list(range(1000, 10000)) + list(range(15000, 30000)) + list(range(35000, 99999))
        ))

    # --- is_vpn_or_proxy: derived SOLELY from ASN ---
    # Datacenter ASN → VPN/proxy detected.  Otherwise → 0.
    vpn_proxy = 1 if asn in DATACENTER_ASN_RANGES else 0


    # --- time_to_book_seconds: how long the session lasted ----
    # Bots/fraud -> 70% very tight around a mean (scripted),
    #              30% wider variance (manual / jittered bots).
    # Legit      -> natural lognormal human browsing time.
    if u in fraud_user_set:
        if np.random.rand() < 0.70:
            ttb = int(np.clip(np.random.normal(45, 5), 15, 90))
        else:
            ttb = int(np.clip(np.random.lognormal(4.2, 0.6), 20, 600))
    else:
        ttb = int(np.clip(np.random.lognormal(5.8, 0.7), 60, 3600))

    session_rows.append({
        "session_id": make_uuid(),
        "booking_id": booking_fact.loc[i, "booking_id"],
        "device_fingerprint": fp,
        "ip_address": ip,
        "asn": asn,
        "is_vpn_or_proxy": vpn_proxy,
        "os": fp_to_os[fp],
        "browser": fp_to_browser[fp],
        "time_to_book_seconds": ttb,
    })

session_context = pd.DataFrame(session_rows)

fp_os_check = session_context.groupby("device_fingerprint")["os"].nunique()
fp_br_check = session_context.groupby("device_fingerprint")["browser"].nunique()
assert (fp_os_check == 1).all(), "FAIL: same fingerprint has different OS values!"
assert (fp_br_check == 1).all(), "FAIL: same fingerprint has different browser values!"

# Verify VPN flag is purely ASN-derived
assert (session_context['is_vpn_or_proxy'] == session_context['asn'].isin(DATACENTER_ASN_RANGES).astype(int)).all(), \
    "FAIL: is_vpn_or_proxy does not match ASN-based derivation!"

print(f"✅ session_context — {len(session_context)} rows")
print(f"   Unique fingerprints: {session_context['device_fingerprint'].nunique()}")
print(f"   Unique IPs: {session_context['ip_address'].nunique()}")
print(f"   Datacenter ASN rate: {session_context['asn'].isin(DATACENTER_ASN_RANGES).mean()*100:.1f}%")
print(f"   VPN/Proxy rate: {session_context['is_vpn_or_proxy'].mean()*100:.1f}% (= datacenter ASN rate)")
print(f"   IP-masked sessions: {ip_masked_count} ({ip_masked_count/N_BOOKINGS*100:.1f}%)")
print(f"   ✅ Fingerprint → OS/Browser consistency verified")
print(f"   ✅ VPN/Proxy = ASN-derived only: verified")

# --- time_to_book_seconds summary ---
_sc_lbl = session_context.merge(
    pd.DataFrame({'booking_id': booking_ids,
                  'is_fraud': [1 if i in fraud_booking_set else 0 for i in range(N_BOOKINGS)]}),
    on='booking_id')
print(f"   Booking time (legit)  — mean: {_sc_lbl[_sc_lbl['is_fraud']==0]['time_to_book_seconds'].mean():.0f}s, "
      f"std: {_sc_lbl[_sc_lbl['is_fraud']==0]['time_to_book_seconds'].std():.0f}s")
print(f"   Booking time (fraud)  — mean: {_sc_lbl[_sc_lbl['is_fraud']==1]['time_to_book_seconds'].mean():.0f}s, "
      f"std: {_sc_lbl[_sc_lbl['is_fraud']==1]['time_to_book_seconds'].std():.0f}s")
session_context.head()

✅ session_context — 8000 rows
   Unique fingerprints: 417
   Unique IPs: 644
   Datacenter ASN rate: 4.9%
   VPN/Proxy rate: 4.9% (= datacenter ASN rate)
   IP-masked sessions: 146 (1.8%)
   ✅ Fingerprint → OS/Browser consistency verified
   ✅ VPN/Proxy = ASN-derived only: verified
   Booking time (legit)  — mean: 427s, std: 342s
   Booking time (fraud)  — mean: 57s, std: 37s


,session_id,booking_id,device_fingerprint,ip_address,asn,is_vpn_or_proxy,os,browser,time_to_book_seconds
0,47d09b95-5666-4a2e-9208-879852f6a39e,B000001,d0057c74a7ffee4b,149.171.156.40,99914,0,Android,Chrome,48
1,aca0bdeb-1a3f-424e-9c3c-37255af07c81,B000002,3ddbdb25ceb330e5,123.162.129.23,44839,0,iOS,Chrome,45
2,e3ab6a14-1a29-4820-9f88-c460fc84add8,B000003,c802ce308ab84b82,5.38.248.97,59354,0,macOS,Firefox,478
3,fda62891-3e6d-4fca-a8e6-da3ccc4a0912,B000004,c1820b45e082fc67,97.60.235.80,84719,0,macOS,Chrome,187
4,dc67d609-7f29-43c4-a6b3-61a6719c5758,B000005,221af74ceee45242,154.132.66.129,36786,0,Android,Chrome,485


In [16]:
# ============================================================
# CELL 5 — PASSENGER DETAILS
# One row per passenger per booking (1:N with booking_fact).
#
# Contains:
#   - passenger_email_domain: the actual email domain given by
#     the passenger during booking (e.g. "travelcorp.com",
#     "gmail.com", "tempmail.xyz")
#
# Fraud signal: legit passengers tend to use the agency's
# corporate domain. Fraud passengers use generic/throwaway.
#
# Note: is_known_employee was removed — in B2B travel, agents
# book for external travelers/clients, not their own employees.
# ============================================================
passenger_rows = []

for i in range(N_BOOKINGS):
    bid = booking_fact.loc[i, "booking_id"]
    uid = booking_fact.loc[i, "user_id"]
    aid = booking_fact.loc[i, "agency_id"]
    n_pax = int(booking_fact.loc[i, "passengers_count"])
    is_fraud_booking = i in fraud_booking_set
    utype = user_master.loc[user_master["user_id"] == uid, "user_fraud_type"].values[0]
    agency_domain = agency_to_domain[aid]

    for p in range(n_pax):
        if is_fraud_booking:
            if utype in ["ring_operator", "new_synthetic_user"]:
                r = np.random.rand()
                if r < 0.10:
                    pax_domain = agency_domain
                elif r < 0.55:
                    pax_domain = np.random.choice(GENERIC_DOMAINS)
                else:
                    pax_domain = np.random.choice(THROWAWAY_DOMAINS)
            elif utype in ["bot_booking", "account_takeover"]:
                r = np.random.rand()
                if r < 0.20:
                    pax_domain = agency_domain
                elif r < 0.70:
                    pax_domain = np.random.choice(GENERIC_DOMAINS)
                else:
                    pax_domain = np.random.choice(THROWAWAY_DOMAINS)
            elif utype == "credit_bustout_user":
                r = np.random.rand()
                if r < 0.35:
                    pax_domain = agency_domain
                elif r < 0.80:
                    pax_domain = np.random.choice(GENERIC_DOMAINS)
                else:
                    pax_domain = np.random.choice(THROWAWAY_DOMAINS)
            else:  # cancellation_abuser
                r = np.random.rand()
                if r < 0.55:
                    pax_domain = agency_domain
                elif r < 0.90:
                    pax_domain = np.random.choice(GENERIC_DOMAINS)
                else:
                    pax_domain = np.random.choice(THROWAWAY_DOMAINS)
        else:
            r = np.random.rand()
            if r < 0.75:
                pax_domain = agency_domain
            elif r < 0.97:
                pax_domain = np.random.choice(GENERIC_DOMAINS)
            else:
                pax_domain = np.random.choice(CORPORATE_DOMAIN_POOL)

        passenger_rows.append({
            "passenger_id": make_uuid(),
            "booking_id": bid,
            "passenger_email_domain": pax_domain,
        })

passenger_details = pd.DataFrame(passenger_rows)

pax_labeled = passenger_details.merge(
    booking_fact[["booking_id"]].assign(is_fraud=booking_fact.index.isin(fraud_booking_set).astype(int)),
    on="booking_id"
)

print(f"✅ passenger_details — {len(passenger_details)} rows  ({N_BOOKINGS} bookings)")
print(f"   Avg passengers/booking: {len(passenger_details)/N_BOOKINGS:.2f}")
print(f"   Columns: {list(passenger_details.columns)}")
print(f"\n   Top email domains (all):")
print(passenger_details['passenger_email_domain'].value_counts().head(10).to_string())
print(f"\n   Top email domains (FRAUD bookings only):")
print(pax_labeled[pax_labeled['is_fraud']==1]['passenger_email_domain'].value_counts().head(10).to_string())
passenger_details.head()

✅ passenger_details — 22003 rows  (8000 bookings)
   Avg passengers/booking: 2.75
   Columns: ['passenger_id', 'booking_id', 'passenger_email_domain']

   Top email domains (all):
passenger_email_domain
skyroute.in       629
gmail.com         569
jetsetgo.in       568
hotmail.com       553
protonmail.com    551
yahoo.com         547
icloud.com        535
travelworks.sg    531
zoho.com          526
aol.com           524

   Top email domains (FRAUD bookings only):
passenger_email_domain
icloud.com         110
protonmail.com     106
zoho.com           100
aol.com             97
throwaway.email     96
yandex.com          93
hotmail.com         89
outlook.com         87
discard.email       84
gmail.com           81


,passenger_id,booking_id,passenger_email_domain
0,e85d67ae-66bb-4117-9278-4e657a5a52c9,B000001,fakeinbox.org
1,c2711c79-ca1f-45e0-b5ae-d379afd000d9,B000001,throwaway.email
2,58751dcd-eaaa-46d7-8a21-4d2c703d225e,B000001,aerolink.uk
3,2805d5e1-657f-4567-8869-60d02277e9e4,B000001,icloud.com
4,bab4315f-cd9b-481d-83ab-5ab0e3df6191,B000001,zoho.com


In [17]:
# ============================================================
# CELL 6 — POST BOOKING EVENTS
# ============================================================
post_events = []

for idx, row in booking_fact.iterrows():
    uid = row["user_id"]
    val = float(row["booking_value"])
    is_fraud_booking = idx in fraud_booking_set
    utype = user_master.loc[user_master["user_id"] == uid, "user_fraud_type"].values[0]

    cancel_p = 0.06
    dispute_p = 0.004

    if is_fraud_booking:
        cancel_p = 0.12
        dispute_p = 0.03

    if utype == "cancellation_abuser":
        cancel_p = 0.70 if not is_fraud_booking else 0.88
        dispute_p = 0.002

    if utype == "credit_bustout_user" and is_fraud_booking:
        cancel_p = 0.07
        dispute_p = 0.18
        booking_fact.loc[idx, "route_type"] = "international"
        booking_fact.loc[idx, "lead_time_days"] = np.random.randint(0, 2)
        booking_fact.loc[idx, "booking_value"] = round(float(np.clip(np.random.normal(18000, 4000), 5000, 25000)), 2)
        booking_fact.loc[idx, "payment_method"] = "credit_line"
        val = float(booking_fact.loc[idx, "booking_value"])

    if utype == "new_synthetic_user" and is_fraud_booking:
        cancel_p = 0.14
        dispute_p = 0.06
        booking_fact.loc[idx, "lead_time_days"] = np.random.randint(0, 4)

    is_cancelled = np.random.rand() < cancel_p
    is_disputed = (not is_cancelled) and (np.random.rand() < dispute_p)

    cancel_delay = int(np.random.randint(0, 3)) if is_cancelled else 0
    dispute_delay = int(np.random.randint(7, 45)) if is_disputed else 0

    chargeback_amount = 0.0
    final_loss_amount = 0.0

    if is_disputed:
        if utype == "credit_bustout_user":
            cb_ratio = np.random.uniform(0.85, 1.00)
            loss_ratio = np.random.uniform(0.65, 0.95)
        elif utype in ["account_takeover", "new_synthetic_user"]:
            cb_ratio = np.random.uniform(0.70, 0.95)
            loss_ratio = np.random.uniform(0.45, 0.85)
        else:
            cb_ratio = np.random.uniform(0.50, 0.90)
            loss_ratio = np.random.uniform(0.25, 0.75)

        chargeback_amount = round(val * cb_ratio, 2)
        final_loss_amount = round(chargeback_amount * loss_ratio, 2)

    post_events.append({
        "event_id": f"E{str(idx+1).zfill(6)}",
        "booking_id": row["booking_id"],
        "is_cancelled": int(is_cancelled),
        "cancel_delay_days": cancel_delay,
        "is_disputed": int(is_disputed),
        "dispute_delay_days": dispute_delay,
        "chargeback_amount": chargeback_amount,
        "final_loss_amount": final_loss_amount
    })

post_booking_events = pd.DataFrame(post_events)

print(f"✅ post_booking_events — {len(post_booking_events)} rows")
print(f"   Cancellations: {post_booking_events['is_cancelled'].sum()}")
print(f"   Disputes: {post_booking_events['is_disputed'].sum()}")
post_booking_events.head()

✅ post_booking_events — 8000 rows
   Cancellations: 578
   Disputes: 61


,event_id,booking_id,is_cancelled,cancel_delay_days,is_disputed,dispute_delay_days,chargeback_amount,final_loss_amount
0,E000001,B000001,0,0,0,0,0.0,0.0
1,E000002,B000002,1,2,0,0,0.0,0.0
2,E000003,B000003,0,0,0,0,0.0,0.0
3,E000004,B000004,0,0,0,0,0.0,0.0
4,E000005,B000005,0,0,0,0,0.0,0.0


In [18]:
# ============================================================
# CELL 7 — BOOKING LABEL TABLE
# ============================================================
booking_label_table = pd.DataFrame({
    "booking_id": booking_fact["booking_id"],
    "fraud_label": 0,
    "fraud_reason": "legit"
})
booking_label_table.loc[list(fraud_booking_set), "fraud_label"] = 1

for idx in fraud_booking_set:
    uid = booking_fact.loc[idx, "user_id"]
    utype = user_master.loc[user_master["user_id"] == uid, "user_fraud_type"].values[0]
    cancelled = post_booking_events.loc[idx, "is_cancelled"] == 1

    if utype == "cancellation_abuser" and cancelled:
        booking_label_table.loc[idx, "fraud_reason"] = "user has abnormally high cancellation rate"
    elif utype == "credit_bustout_user":
        booking_label_table.loc[idx, "fraud_reason"] = "credit bustout: high value international + high loss"
    elif utype == "new_synthetic_user":
        booking_label_table.loc[idx, "fraud_reason"] = "new user with risky infra + abnormal velocity"
    elif utype == "account_takeover":
        booking_label_table.loc[idx, "fraud_reason"] = "new device/ip + short lead time"
    elif utype == "ring_operator":
        booking_label_table.loc[idx, "fraud_reason"] = "shared infra across multiple fraud users"
    elif utype == "bot_booking":
        booking_label_table.loc[idx, "fraud_reason"] = "burst/automation pattern in activity"
    else:
        booking_label_table.loc[idx, "fraud_reason"] = "fraud user suspicious booking pattern"

print(f"✅ booking_label_table — {len(booking_label_table)} rows")
print(f"   Fraud bookings: {booking_label_table['fraud_label'].sum()}")
print(booking_label_table[booking_label_table['fraud_label']==1]['fraud_reason'].value_counts())
booking_label_table.head()

✅ booking_label_table — 8000 rows
   Fraud bookings: 720
fraud_reason
shared infra across multiple fraud users                140
new device/ip + short lead time                         140
credit bustout: high value international + high loss    133
burst/automation pattern in activity                    112
new user with risky infra + abnormal velocity           107
user has abnormally high cancellation rate               78
fraud user suspicious booking pattern                    10
Name: count, dtype: int64


,booking_id,fraud_label,fraud_reason
0,B000001,1,shared infra across multiple fraud users
1,B000002,1,credit bustout: high value international + hig...
2,B000003,0,legit
3,B000004,0,legit
4,B000005,0,legit


In [19]:
# ============================================================
# CELL 8 — SAVE FILES & SANITY CHECKS
# ============================================================
agency_master.to_csv(os.path.join(OUT_DIR, "agency_master.csv"), index=False)
user_master.to_csv(os.path.join(OUT_DIR, "user_master.csv"), index=False)
booking_fact.to_csv(os.path.join(OUT_DIR, "booking_fact.csv"), index=False)
session_context.to_csv(os.path.join(OUT_DIR, "session_context.csv"), index=False)
passenger_details.to_csv(os.path.join(OUT_DIR, "passenger_details.csv"), index=False)
post_booking_events.to_csv(os.path.join(OUT_DIR, "post_booking_events.csv"), index=False)
booking_label_table.to_csv(os.path.join(OUT_DIR, "booking_label_table.csv"), index=False)

for old_file in ["device_master.csv", "ip_master.csv"]:
    old_path = os.path.join(OUT_DIR, old_file)
    if os.path.exists(old_path):
        os.remove(old_path)
        print(f"🗑️  Removed old {old_file}")



print("\n✅ Dataset generated!")
print("Folder:", OUT_DIR)

print("\n" + "="*80)
print("  SANITY CHECKS")
print("="*80)

# ------------------------------------------------------------------
# SECTION A — BASIC STRUCTURAL CHECKS
# ------------------------------------------------------------------
print("\n--- A. Basic Structural Checks ---")

assert 'device_id' not in booking_fact.columns
assert 'ip_id' not in booking_fact.columns
print("✅ booking_fact has no device_id / ip_id")

assert len(session_context) == N_BOOKINGS
fp_os_check = session_context.groupby("device_fingerprint")["os"].nunique()
fp_br_check = session_context.groupby("device_fingerprint")["browser"].nunique()
assert (fp_os_check == 1).all(), "FAIL: same fingerprint has different OS!"
assert (fp_br_check == 1).all(), "FAIL: same fingerprint has different browser!"
print("✅ Fingerprint → OS/Browser consistency: PASSED")

assert (session_context['is_vpn_or_proxy'] == session_context['asn'].isin(DATACENTER_ASN_RANGES).astype(int)).all()
print("✅ VPN/Proxy = ASN-derived only: PASSED")

assert len(passenger_details) >= N_BOOKINGS
assert 'passenger_email_domain' in passenger_details.columns
assert 'email_domain_match_flag' not in passenger_details.columns
assert 'is_known_employee' not in passenger_details.columns
assert 'email_domain_match_flag' in user_master.columns
assert 'agency_email_domain' in agency_master.columns
print("✅ Column placement: PASSED")

# ------------------------------------------------------------------
# SECTION B — REFERENTIAL INTEGRITY (no orphan keys)
# ------------------------------------------------------------------
print("\n--- B. Referential Integrity (no orphan keys) ---")

# Every booking_id in booking_fact exists in booking_label_table and vice-versa
bf_ids = set(booking_fact["booking_id"])
bl_ids = set(booking_label_table["booking_id"])
assert bf_ids == bl_ids, f"FAIL: booking_fact ↔ booking_label_table mismatch! Only in fact: {bf_ids - bl_ids}, Only in label: {bl_ids - bf_ids}"
print("✅ booking_fact ↔ booking_label_table: booking_id sets match exactly")

# Every booking_id in session_context exists in booking_fact
sc_ids = set(session_context["booking_id"])
assert sc_ids == bf_ids, f"FAIL: session_context ↔ booking_fact mismatch! Only in session: {sc_ids - bf_ids}, Only in fact: {bf_ids - sc_ids}"
print("✅ session_context ↔ booking_fact: booking_id sets match exactly")

# Every booking_id in post_booking_events exists in booking_fact
pb_ids = set(post_booking_events["booking_id"])
assert pb_ids == bf_ids, f"FAIL: post_booking_events ↔ booking_fact mismatch! Only in post: {pb_ids - bf_ids}, Only in fact: {bf_ids - pb_ids}"
print("✅ post_booking_events ↔ booking_fact: booking_id sets match exactly")

# Every booking_id in passenger_details exists in booking_fact
pax_bids = set(passenger_details["booking_id"])
assert pax_bids.issubset(bf_ids), f"FAIL: passenger_details has booking_ids not in booking_fact: {pax_bids - bf_ids}"
assert bf_ids.issubset(pax_bids), f"FAIL: booking_fact has booking_ids with no passengers: {bf_ids - pax_bids}"
print("✅ passenger_details ↔ booking_fact: every booking has passengers, no orphans")

# Every user_id in booking_fact exists in user_master
bf_uids = set(booking_fact["user_id"])
um_uids = set(user_master["user_id"])
assert bf_uids.issubset(um_uids), f"FAIL: booking_fact has user_ids not in user_master: {bf_uids - um_uids}"
print("✅ booking_fact → user_master: all user_ids valid")

# Every agency_id in booking_fact exists in agency_master
bf_aids = set(booking_fact["agency_id"])
am_aids = set(agency_master["agency_id"])
assert bf_aids.issubset(am_aids), f"FAIL: booking_fact has agency_ids not in agency_master: {bf_aids - am_aids}"
print("✅ booking_fact → agency_master: all agency_ids valid")

# Every agency_id in user_master exists in agency_master
um_aids = set(user_master["agency_id"])
assert um_aids.issubset(am_aids), f"FAIL: user_master has agency_ids not in agency_master: {um_aids - am_aids}"
print("✅ user_master → agency_master: all agency_ids valid")

# ------------------------------------------------------------------
# SECTION C — CROSS-TABLE FRAUD LABEL CONSISTENCY
# ------------------------------------------------------------------
print("\n--- C. Cross-Table Fraud Label Consistency ---")

# C1: Every fraud booking must belong to a fraud user
fraud_bids = booking_label_table[booking_label_table["fraud_label"] == 1]["booking_id"].values
fraud_booking_users = booking_fact[booking_fact["booking_id"].isin(fraud_bids)]["user_id"].unique()
fraud_uids_from_um = set(user_master[user_master["user_fraud_label"] == 1]["user_id"])
bad_users = set(fraud_booking_users) - fraud_uids_from_um
assert len(bad_users) == 0, f"FAIL: {len(bad_users)} fraud bookings belong to NON-fraud users: {bad_users}"
print(f"✅ C1: All {len(fraud_bids)} fraud bookings belong to fraud-labeled users")

# C2: Check if any good bookings belong to fraud users (by design this can happen)
good_bids = booking_label_table[booking_label_table["fraud_label"] == 0]["booking_id"].values
good_booking_users = booking_fact[booking_fact["booking_id"].isin(good_bids)]["user_id"].unique()
good_uids_from_um = set(user_master[user_master["user_fraud_label"] == 0]["user_id"])
good_bookings_by_fraud_users = set(good_booking_users) - good_uids_from_um
if len(good_bookings_by_fraud_users) > 0:
    good_by_fraud = booking_fact[
        (booking_fact["booking_id"].isin(good_bids)) &
        (booking_fact["user_id"].isin(good_bookings_by_fraud_users))
    ]
    print(f"⚠️  C2: {len(good_bookings_by_fraud_users)} fraud users also have GOOD bookings (by design)")
    print(f"      → {len(good_by_fraud)} good bookings from fraud users (out of {len(good_bids)} total good)")
else:
    print("✅ C2: All good bookings belong to good (legit) users")

# C3: Agency linkage consistency — booking_fact.agency_id must match
#     the agency_id of the user in user_master for that booking
bf_check = booking_fact[["booking_id", "user_id", "agency_id"]].merge(
    user_master[["user_id", "agency_id"]], on="user_id", suffixes=("_booking", "_user")
)
agency_mismatch = bf_check[bf_check["agency_id_booking"] != bf_check["agency_id_user"]]
assert len(agency_mismatch) == 0, (
    f"FAIL: {len(agency_mismatch)} bookings have agency_id mismatch between "
    f"booking_fact and user_master!\n{agency_mismatch.head(5).to_string()}"
)
print(f"✅ C3: Agency linkage consistent — booking_fact.agency_id matches "
      f"user_master.agency_id for all {len(bf_check)} bookings")

# C4: fraud_reason in booking_label_table aligns with user_fraud_type in user_master
reason_type_map = {
    "user has abnormally high cancellation rate": "cancellation_abuser",
    "credit bustout: high value international + high loss": "credit_bustout_user",
    "new user with risky infra + abnormal velocity": "new_synthetic_user",
    "new device/ip + short lead time": "account_takeover",
    "shared infra across multiple fraud users": "ring_operator",
    "burst/automation pattern in activity": "bot_booking",
}
fraud_labels_with_user = booking_label_table[booking_label_table["fraud_label"] == 1].merge(
    booking_fact[["booking_id", "user_id"]], on="booking_id"
).merge(
    user_master[["user_id", "user_fraud_type"]], on="user_id"
)
mismatches = []
for _, row in fraud_labels_with_user.iterrows():
    expected_type = reason_type_map.get(row["fraud_reason"])
    if expected_type is not None and expected_type != row["user_fraud_type"]:
        mismatches.append({
            "booking_id": row["booking_id"],
            "fraud_reason": row["fraud_reason"],
            "expected_type": expected_type,
            "actual_type": row["user_fraud_type"]
        })
if len(mismatches) == 0:
    print(f"✅ C4: fraud_reason ↔ user_fraud_type aligned for all "
          f"{len(fraud_labels_with_user)} fraud bookings")
else:
    print(f"❌ C4 FAIL: {len(mismatches)} bookings have fraud_reason ↔ "
          f"user_fraud_type mismatch!")
    for m in mismatches[:5]:
        print(f"   {m}")
    assert False, "Cross-table fraud_reason ↔ user_fraud_type mismatch detected!"

# ------------------------------------------------------------------
# SECTION D — POST-BOOKING EVENTS CONSISTENCY WITH FRAUD TYPE
# ------------------------------------------------------------------
print("\n--- D. Post-Booking Events ↔ Fraud Type Consistency ---")

pbe_labeled = post_booking_events.merge(
    booking_label_table[["booking_id", "fraud_label", "fraud_reason"]], on="booking_id"
).merge(
    booking_fact[["booking_id", "user_id"]], on="booking_id"
).merge(
    user_master[["user_id", "user_fraud_type"]], on="user_id"
)

# D1: cancellation_abuser fraud bookings should have higher cancel rate
cancel_abuse_fraud = pbe_labeled[
    (pbe_labeled["user_fraud_type"] == "cancellation_abuser") &
    (pbe_labeled["fraud_label"] == 1)
]
overall_cancel_rate = pbe_labeled["is_cancelled"].mean()
if len(cancel_abuse_fraud) > 0:
    ca_cancel_rate = cancel_abuse_fraud["is_cancelled"].mean()
    print(f"   D1: cancellation_abuser (fraud) cancel rate: "
          f"{ca_cancel_rate*100:.1f}% vs overall: {overall_cancel_rate*100:.1f}%")
    assert ca_cancel_rate > overall_cancel_rate, \
        "FAIL: cancellation_abuser fraud bookings should have higher cancel rate!"
    print(f"✅ D1: cancellation_abuser fraud bookings have elevated cancel rate")

# D2: credit_bustout_user fraud bookings should have higher dispute rate
bustout_fraud = pbe_labeled[
    (pbe_labeled["user_fraud_type"] == "credit_bustout_user") &
    (pbe_labeled["fraud_label"] == 1)
]
overall_dispute_rate = pbe_labeled["is_disputed"].mean()
if len(bustout_fraud) > 0:
    cb_dispute_rate = bustout_fraud["is_disputed"].mean()
    print(f"   D2: credit_bustout_user (fraud) dispute rate: "
          f"{cb_dispute_rate*100:.1f}% vs overall: {overall_dispute_rate*100:.1f}%")
    assert cb_dispute_rate > overall_dispute_rate, \
        "FAIL: credit_bustout_user fraud bookings should have higher dispute rate!"
    print(f"✅ D2: credit_bustout_user fraud bookings have elevated dispute rate")

# D3: Good bookings should have lower cancel + dispute rates than fraud bookings
fraud_pbe = pbe_labeled[pbe_labeled["fraud_label"] == 1]
good_pbe = pbe_labeled[pbe_labeled["fraud_label"] == 0]
fraud_cancel = fraud_pbe["is_cancelled"].mean()
good_cancel = good_pbe["is_cancelled"].mean()
fraud_dispute = fraud_pbe["is_disputed"].mean()
good_dispute = good_pbe["is_disputed"].mean()
print(f"   D3: Cancel rate  — Fraud: {fraud_cancel*100:.1f}% vs Good: {good_cancel*100:.1f}%")
print(f"   D3: Dispute rate — Fraud: {fraud_dispute*100:.1f}% vs Good: {good_dispute*100:.1f}%")

# ------------------------------------------------------------------
# SECTION E — SESSION CONTEXT ↔ FRAUD LABEL CONSISTENCY
# ------------------------------------------------------------------
print("\n--- E. Session Context ↔ Fraud Label Consistency ---")

sc_with_label = session_context.merge(
    booking_label_table[["booking_id", "fraud_label"]], on="booking_id"
).merge(
    booking_fact[["booking_id", "user_id"]], on="booking_id"
).merge(
    user_master[["user_id", "user_fraud_type"]], on="user_id"
)

# E1: Fraud bookings should have higher VPN/proxy rate
fraud_vpn = sc_with_label[sc_with_label["fraud_label"] == 1]["is_vpn_or_proxy"].mean()
good_vpn = sc_with_label[sc_with_label["fraud_label"] == 0]["is_vpn_or_proxy"].mean()
print(f"   E1: VPN/Proxy rate — Fraud: {fraud_vpn*100:.1f}% vs Good: {good_vpn*100:.1f}%")
assert fraud_vpn > good_vpn, "FAIL: Fraud bookings should have higher VPN/proxy rate!"
print("✅ E1: Fraud bookings have elevated VPN/proxy rate")

# E2: Avg unique IPs per device — fraud should be higher (IP masking signal)
fraud_ip_per_fp = sc_with_label[sc_with_label['fraud_label']==1] \
    .groupby('device_fingerprint')['ip_address'].nunique().mean()
legit_ip_per_fp = sc_with_label[sc_with_label['fraud_label']==0] \
    .groupby('device_fingerprint')['ip_address'].nunique().mean()
print(f"   E2: Avg unique IPs per device — Fraud: {fraud_ip_per_fp:.2f}, Good: {legit_ip_per_fp:.2f}")
assert fraud_ip_per_fp > legit_ip_per_fp, \
    "FAIL: Fraud should have more IPs per device (IP masking)!"
print("✅ E2: Fraud has higher IP diversity per device (IP masking detected)")

# E3: Booking time — fraud should be faster (bots/scripts)
fraud_ttb = sc_with_label[sc_with_label['fraud_label']==1]['time_to_book_seconds'].mean()
good_ttb = sc_with_label[sc_with_label['fraud_label']==0]['time_to_book_seconds'].mean()
print(f"   E3: Avg booking time — Fraud: {fraud_ttb:.0f}s vs Good: {good_ttb:.0f}s")
assert fraud_ttb < good_ttb, "FAIL: Fraud bookings should be faster (bot-like)!"
print("✅ E3: Fraud bookings have lower booking time (bot/script pattern detected)")

# ------------------------------------------------------------------
# SECTION F — PASSENGER DETAILS ↔ FRAUD LABEL CONSISTENCY
# ------------------------------------------------------------------
print("\n--- F. Passenger Details ↔ Fraud Label Consistency ---")

pax_labeled = passenger_details.merge(
    booking_label_table[["booking_id", "fraud_label"]], on="booking_id"
).merge(
    booking_fact[["booking_id", "agency_id"]], on="booking_id"
).merge(
    agency_master[["agency_id", "agency_email_domain"]], on="agency_id"
)

# F1: Corporate domain match rate — legit passengers should match agency domain more
pax_labeled["domain_matches_agency"] = (
    pax_labeled["passenger_email_domain"] == pax_labeled["agency_email_domain"]
).astype(int)
fraud_domain_match = pax_labeled[pax_labeled["fraud_label"] == 1]["domain_matches_agency"].mean()
good_domain_match = pax_labeled[pax_labeled["fraud_label"] == 0]["domain_matches_agency"].mean()
print(f"   F1: Passenger email matches agency domain — "
      f"Fraud: {fraud_domain_match*100:.1f}% vs Good: {good_domain_match*100:.1f}%")
assert good_domain_match > fraud_domain_match, \
    "FAIL: Good bookings should have higher agency domain match rate!"
print("✅ F1: Good bookings have higher agency-email domain match")

# F2: Throwaway email usage — fraud passengers should use more throwaway domains
throwaway_set = set(THROWAWAY_DOMAINS)
pax_labeled["is_throwaway"] = pax_labeled["passenger_email_domain"].isin(throwaway_set).astype(int)
fraud_throwaway = pax_labeled[pax_labeled["fraud_label"] == 1]["is_throwaway"].mean()
good_throwaway = pax_labeled[pax_labeled["fraud_label"] == 0]["is_throwaway"].mean()
print(f"   F2: Throwaway email usage — "
      f"Fraud: {fraud_throwaway*100:.1f}% vs Good: {good_throwaway*100:.1f}%")
assert fraud_throwaway > good_throwaway, \
    "FAIL: Fraud bookings should have higher throwaway email usage!"
print("✅ F2: Fraud bookings have higher throwaway email usage")

# ------------------------------------------------------------------
# SECTION G — SUMMARY STATS
# ------------------------------------------------------------------
print("\n--- G. Summary Statistics ---")
print(f"  Tables generated: 7")
print(f"  agency_master:       {len(agency_master):>6} rows")
print(f"  user_master:         {len(user_master):>6} rows  "
      f"(fraud: {user_master['user_fraud_label'].sum()})")
print(f"  booking_fact:        {len(booking_fact):>6} rows")
print(f"  session_context:     {len(session_context):>6} rows")
print(f"  passenger_details:   {len(passenger_details):>6} rows")
print(f"  post_booking_events: {len(post_booking_events):>6} rows")
print(f"  booking_label_table: {len(booking_label_table):>6} rows  "
      f"(fraud: {booking_label_table['fraud_label'].sum()})")
print(f"\n{'='*80}")
print("  ✅ ALL CROSS-TABLE FRAUD LABEL CONSISTENCY CHECKS PASSED")
print(f"{'='*80}")



✅ Dataset generated!
Folder: /Users/sumitsaurabh/fraud_detection/b2b-travel-fraud-detection/Data/Tables

  SANITY CHECKS

--- A. Basic Structural Checks ---
✅ booking_fact has no device_id / ip_id
✅ Fingerprint → OS/Browser consistency: PASSED
✅ VPN/Proxy = ASN-derived only: PASSED
✅ Column placement: PASSED

--- B. Referential Integrity (no orphan keys) ---
✅ booking_fact ↔ booking_label_table: booking_id sets match exactly
✅ session_context ↔ booking_fact: booking_id sets match exactly
✅ post_booking_events ↔ booking_fact: booking_id sets match exactly
✅ passenger_details ↔ booking_fact: every booking has passengers, no orphans
✅ booking_fact → user_master: all user_ids valid
✅ booking_fact → agency_master: all agency_ids valid
✅ user_master → agency_master: all agency_ids valid

--- C. Cross-Table Fraud Label Consistency ---
✅ C1: All 720 fraud bookings belong to fraud-labeled users
✅ C2: All good bookings belong to good (legit) users
✅ C3: Agency linkage consistent — booking_fact.